In [18]:
import av

from PIL import Image, ImageEnhance
from matplotlib import pyplot as plt

import tensorflow as tf

import keras
from keras import layers
from keras.optimizers import Adam
from keras.applications.xception import Xception

import os
import random
import math
import numpy as np
import pickle

In [2]:
DS_CDFV1 = 'celeb_df_v1/'
DS_CDFV2 = 'celeb_df_v2/'

DS_ORGINAL = 'dataset_original/'
DS_SPLIT = 'dataset_split/'
DS_IFRAMES = 'dataset_iframes/'
DS_FACE = 'dataset_face/'
DS_FACE_IMG = 'dataset_face_img/'
DS_SRM_SNIPPETS = 'dataset_srm_snippets_5/'
DS_SEGMENTS = 'dataset_segments/'
DS_RAW = 'dataset_raw/'
DS_RESIDUALS = 'dataset_residuals/'
DS_TEMPORAL = 'dataset_temporal/'


SEG_1 = 'seg_1/'
SEG_2 = 'seg_2/'
SEG_3 = 'seg_3/'
SEG_4 = 'seg_4/'
SEG_5 = 'seg_5/'

SEG = ['seg_1_', 'seg_2_', 'seg_3_', 'seg_4_', 'seg_5_']

DS_TRAIN = 'train_dataset/'
DS_TEST = 'test_dataset/'
DS_VAL = 'val_dataset/'

CLASS_FAKE = 'fake/'
CLASS_REAL = 'real/'


TOP_LEVEL_1 = [DS_SPLIT, DS_IFRAMES, DS_FACE, DS_FACE_IMG, DS_SRM_SNIPPETS]
TOP_LEVEL_2 = [DS_SEGMENTS, DS_RAW, DS_RESIDUALS]
SEGMENTS = [SEG_1, SEG_2, SEG_3, SEG_4, SEG_5]
SPLIT = [DS_TRAIN, DS_TEST, DS_VAL]
CLASS = [CLASS_REAL, CLASS_FAKE]

DATASET = [DS_CDFV1, DS_CDFV2]

In [3]:
SAVE_CHECKPOINT_PATH = 'models/SRM_Model/checkpoints/'
SAVE_METRICS_PATH = 'models/SRM_Model/metrics/'
SAVE_MODEL_PATH = 'models/SRM_Model/final_model/'
BACKUP_MODEL_PATH = 'models/SRM_Model/backups/'

TEST_SCORE_PATH = 'test_scores/'

# Snippet Extraction

## Functions

In [4]:
# Returns the index of frames that begin a new segment (except the first segment)
def get_segment_dividers(frame_count, num_segments):
    segments_per_frame = math.floor(frame_count / num_segments)

    return [(segments_per_frame * i) for i in range(1, num_segments) ]

In [5]:
# Returns the indices of the frames that will be randomly selected from each segment
# Multiple snippets indices per segment can be returned by setting the num_snippets arg 
def get_snippet_indices(segment_dividers, num_snippets):
    start_index = 0
    num_snippets = 1 if num_snippets <= 0 else num_snippets

    snippet_indices = []
    for end_index in segment_dividers:

        # Extracting multiple snippets per segment (if needed)
        for _ in range(num_snippets):
            snippet_indices.append(random.randint(start_index, end_index - 1))

        start_index = end_index
        
    return snippet_indices

In [6]:
# Returns an array of randomly selected snippets(PIL.Image) from each segment of the input video
def extract_snippets(fp, num_segments, num_snippets):
    vid_container = av.open(fp)
    vid_stream = vid_container.streams.video[0]
    frame_count = vid_stream.frames

    snippets = []

    # If number of frames in video is less than the number of frames that need to sampled
    # then take all frames in the video
    if frame_count < num_segments * num_snippets:
        for frame in vid_container.decode():
            snippets.append(frame.to_image())

    else:
        segment_dividers = get_segment_dividers(frame_count, num_segments)
        segment_dividers = segment_dividers + [frame_count]

        snippet_indices = get_snippet_indices(segment_dividers, num_snippets)

        frame_index = 0
        for frame in vid_container.decode():
            if frame_index > max(snippet_indices):
                break

            if frame_index in snippet_indices:
                snippets.append(frame.to_image())

            frame_index += 1

    return snippets

## Implementation

### Celeb-DF v1 & v2

In [103]:
def save_snippets_CDF(dataset, num_segments, num_snippets):
    if dataset != DS_CDFV1 and dataset != DS_CDFV2:
        print(dataset)
        return
    
    random.seed(1)
    
    src_base_path = dataset + DS_FACE
    dst_base_path = dataset + DS_SRM_SNIPPETS

    for split in SPLIT:
        print(f'---Split started: {split}---')
        for class_dir in CLASS:
            print(f'Class started: {class_dir}')

            for video in os.listdir(src_base_path + split + class_dir):
                fp = src_base_path + split + class_dir + video
                snippets = extract_snippets(fp, num_segments, num_snippets)

                for i, snippet in enumerate(snippets, start=1):
                    seg_index = math.ceil(float(i) / num_snippets)
                    snip_index = (i - 1) % num_snippets
              
                    dst = f'{dst_base_path + split + class_dir + os.path.splitext(video)[0]}_s{seg_index}_f{snip_index}.jpeg'
                    snippet.save(dst)         

In [85]:
# CELEB DF V1
save_snippets_CDF(DS_CDFV1, num_segments=5, num_snippets=1)

---Split started: train_dataset/---
Class started: real/
Class started: fake/
---Split started: test_dataset/---
Class started: real/
Class started: fake/
---Split started: val_dataset/---
Class started: real/
Class started: fake/


In [104]:
# CELEB DF V2
save_snippets_CDF(DS_CDFV2, num_segments=5, num_snippets=1)

---Split started: train_dataset/---
Class started: real/
Class started: fake/
---Split started: test_dataset/---
Class started: real/
Class started: fake/
---Split started: val_dataset/---
Class started: real/
Class started: fake/


## Testing Logic

In [ ]:
tmp_count = 30
tmp_seg = get_segment_dividers(tmp_count, 3)
tmp_snip = get_snippet_indices(tmp_seg + [tmp_count], 2)

print(f'Segment Dividers: {tmp_seg}')
print(f'Snippets{tmp_snip}')

Segment Dividers: [10, 20]
Snippets[4, 9, 15, 13, 20, 26]


In [ ]:
# test_file = os.listdir(DS_CDFV1 + DS_SPLIT + DS_TRAIN + CLASS_REAL)[0]
test_file = DS_CDFV2 + DS_SPLIT + DS_TRAIN + CLASS_REAL + 'id27_0005.mp4'
# test_input = av.open(os.path.realpath(DS_CDFV1 + DS_SPLIT + DS_TRAIN + CLASS_REAL + test_file))
test_input = av.open(test_file)

print(test_input.streams.video[0].frames)

# for frame in test_input.decode():
#     print(frame.key_frame)

1


In [ ]:
# test_file = os.listdir(DS_CDFV1 + DS_FACE + DS_TRAIN + CLASS_REAL)[0]
test_file = DS_CDFV2 + DS_FACE + DS_TRAIN + CLASS_REAL + 'id27_0005.mp4'
# test_file = DS_CDFV1 + DS_FACE + DS_TRAIN + CLASS_REAL + test_file
tmp_snippets = extract_snippets(test_file, 5, 1)

for s in tmp_snippets:
    s.show()

# Data Augmentation (Train)

In [16]:
def flip_and_save_image(dataset, split, class_dir):
    img_dir = dataset + DS_SRM_SNIPPETS + split + class_dir

    count = 0
    files = os.listdir(img_dir)
    for file in files:
        image = Image.open(img_dir + file)
        image_flipped = image.transpose(Image.FLIP_LEFT_RIGHT)

        image_flipped.save(img_dir + os.path.splitext(file)[0] + '_flip.jpeg')
        count += 1

    print(f'{len(files)} files found.')
    print(f'{count} files flipped.')

In [21]:
def increase_brightness(dataset, split, class_dir, factor):
    img_dir = dataset + DS_SRM_SNIPPETS + split + class_dir
    for file in os.listdir(img_dir):
        image = Image.open(img_dir + file)

        enhancer = ImageEnhance.Brightness(image)
        image_enhanced = enhancer.enhance(factor)

        image_enhanced.save(img_dir + os.path.splitext(file)[0] + '_enhanced.jpeg')


In [ ]:
flip_and_save_image(DS_CDFV2, DS_TRAIN, CLASS_REAL)

In [23]:
increase_brightness(DS_CDFV2, DS_TRAIN, CLASS_REAL, 1.5)

# Tensor Dataset Creation

In [24]:
def create_tensor_dataset(dataset, split):
    ds = keras.utils.image_dataset_from_directory(
        directory = dataset + DS_SRM_SNIPPETS + split,
        labels = 'inferred',
        label_mode = 'binary',
        batch_size = 32,
        color_mode = 'rgb',
        shuffle = True,
        seed = 1
    )

    return ds

## Celeb DF v1

In [54]:
train_dataset_cdfv1 = create_tensor_dataset(DS_CDFV1, DS_TRAIN)
test_dataset_cdfv1 = create_tensor_dataset(DS_CDFV1, DS_TEST)
val_dataset_cdfv1 = create_tensor_dataset(DS_CDFV1, DS_VAL)

Found 4415 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Found 1100 files belonging to 2 classes.


## Celeb DF v2

In [25]:
train_dataset_cdfv2 = create_tensor_dataset(DS_CDFV2, DS_TRAIN)
test_dataset_cdfv2 = create_tensor_dataset(DS_CDFV2, DS_TEST)
val_dataset_cdfv2 = create_tensor_dataset(DS_CDFV2, DS_VAL)

Found 32584 files belonging to 2 classes.
Found 2590 files belonging to 2 classes.
Found 6005 files belonging to 2 classes.


# Model Creation

**In Convolution**
- Input Shape:  [batch_size, height, width, channels]
- Filter Shape: [height, width, in_channels, filter_count]

## Functions

In [37]:
class SRMLayer(keras.layers.Layer):
    def __init__(self, strides=[1,1,1,1], padding='SAME'):
        super(SRMLayer, self).__init__()
        self.strides = strides
        self.padding = padding

        # Set of 3 fixed SRM Filters used to extract noise & semantic features
        self.filter_small = tf.constant([[0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 1, -2, 1, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0]], dtype=tf.float32)
        
        self.filter_med = tf.constant([[0,  0,  0,  0, 0],
                                       [0, -1,  2, -1, 0],
                                       [0,  2, -4,  2, 0],
                                       [0, -1,  2, -1, 0],
                                       [0,  0,  0,  0, 0]], dtype=tf.float32)
        
        self.filter_large = tf.constant([[-1,  2,  -2,  2, -1],
                                         [ 2, -6,   8, -6,  2],
                                         [-2,  8, -12,  8, -2],
                                         [ 2, -6,   8, -6,  2],
                                         [-1,  2,  -2,  2, -1]], dtype=tf.float32)

        # Learnability in SRM filters introduced by 'q' values
        # SRM filters are divided by their respective 'q' values before convolution
        # 'q' values are updated during backpropagation using gradient descent
        self.q_small = self.add_weight(name='q_small',
                                       shape=(5, 5, 3, 1),
                                       initializer=keras.initializers.Constant(value=2.0),
                                       trainable=True)
        
        self.q_med = self.add_weight(name='q_med',
                                     shape=(5, 5, 3, 1),
                                     initializer=keras.initializers.Constant(value=4.0),
                                     trainable=True)
        
        self.q_large = self.add_weight(name='q_large',
                                       shape=(5, 5, 3, 1),
                                       initializer=keras.initializers.Constant(value=12.0),
                                       trainable=True)
        
        # 3rd dimension of filters => number of input channels (Three channels)
        self.filter_small = tf.stack([self.filter_small, self.filter_small, self.filter_small], axis=2)
        self.filter_med   = tf.stack([self.filter_med, self.filter_med, self.filter_med], axis=2)
        self.filter_large = tf.stack([self.filter_large, self.filter_large, self.filter_large], axis=2)

        # 4th dimension of filters => number of output feature maps (One feature map)
        # Each filter gives a single output feature map
        self.filter_small = tf.expand_dims(self.filter_small, axis=-1)
        self.filter_med   = tf.expand_dims(self.filter_med, axis=-1)
        self.filter_large = tf.expand_dims(self.filter_large, axis=-1)
        
    def call(self, inputs):
        filter_small = tf.math.divide(self.filter_small, self.q_small)
        filter_med   = tf.math.divide(self.filter_med, self.q_med)
        filter_large = tf.math.divide(self.filter_large, self.q_large)

        output_small = tf.nn.conv2d(inputs, filter_small, strides=self.strides, padding=self.padding)
        output_med   = tf.nn.conv2d(inputs, filter_med,   strides=self.strides, padding=self.padding)
        output_large = tf.nn.conv2d(inputs, filter_large, strides=self.strides, padding=self.padding)

        return tf.concat([output_small, output_med, output_large], axis=3)

    def get_config(self):
        config = super(SRMLayer, self).get_config()
        config.update({'strides': self.strides,
                       'padding': self.padding,
                       'filter_small': self.filter_small,
                       'filter_med': self.filter_med,
                       'filter_large': self.filter_large,
                       'q_small': self.q_small,
                       'q_med': self.q_med,
                       'q_large': self.q_large})
        
        return config


In [38]:
XceptionNetwork = keras.applications.Xception(
    include_top = False,
    weights = 'imagenet',
    input_shape = (256, 256, 3),
    pooling = max,
    classes = 2
)

In [40]:
class MetricsCheckPoint(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):

        metrics_dict = {}
        if os.path.exists(SAVE_METRICS_PATH + 'metrics.pkl'):
            with open(SAVE_METRICS_PATH + 'metrics.pkl', 'rb') as f:
                metrics_dict = pickle.load(f)

        for key, value in logs.items():
            if key not in metrics_dict:
                metrics_dict[key] = []

            metrics_dict[key].append(value)

        with open(SAVE_METRICS_PATH + 'metrics.pkl', 'wb') as f:
            pickle.dump(metrics_dict, f)

In [41]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = SAVE_CHECKPOINT_PATH + 'checkpoint',
    save_weights_only = True
)

In [42]:
early_stopping_callback = keras.callbacks.EarlyStopping(
    patience = 5,
    restore_best_weights = True,
    start_from_epoch = 50
)

In [43]:
model_restore_callback = keras.callbacks.BackupAndRestore(
    backup_dir = BACKUP_MODEL_PATH + 'backup'
)

## Implementation

### Model Creation

In [47]:
# Ensuring all layers are trainable (fine-tuning)
XceptionNetwork.trainable = True

# TODO: Freezing all BN layers except for first one (can be done better)
# Freeze all BN layers
for layer in XceptionNetwork.layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

# Unfreeze first BN layer
for layer in XceptionNetwork.layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = True
        break

In [48]:
seen_first = False
for layer in XceptionNetwork.layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        if seen_first == False:
            assert layer.trainable == True
            seen_first = True
        else:
            assert layer.trainable == False

In [49]:
def create_SRM_model(xception_training):
    inputs = keras.layers.Input(shape=(256, 256, 3))
    SRM_noise_maps = SRMLayer()(inputs)
    
    feature_maps = tf.keras.applications.xception.preprocess_input(SRM_noise_maps)
    feature_maps = XceptionNetwork(feature_maps, training=xception_training)

    features = keras.layers.Flatten()(feature_maps)
    features = keras.layers.Dropout(0.8)(features)
    features = keras.layers.Dense(units=130, activation=keras.layers.LeakyReLU())(features)
    outputs = keras.layers.Dense(units=1, activation='sigmoid')(features)

    return keras.Model(inputs, outputs, name='SRM_Model')

In [50]:
SRM_Model = create_SRM_model(xception_training=False)

In [51]:
optimizer = keras.optimizers.Adam(beta_1=0.9, 
                                  beta_2=0.999, 
                                  epsilon=1e-6, 
                                  learning_rate=0.00002)

SRM_Model.compile(optimizer=optimizer,
                      loss=keras.losses.BinaryCrossentropy(),
                      metrics=[keras.metrics.BinaryAccuracy(), 
                               keras.metrics.Precision(), 
                               keras.metrics.Recall(),
                               keras.metrics.AUC(),
                               keras.metrics.FalseNegatives(),
                               keras.metrics.FalsePositives(),
                               keras.metrics.TrueNegatives(),
                               keras.metrics.TruePositives()])

SRM_Model.summary()

Model: "SRM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 srm_layer_1 (SRMLayer)      (None, 256, 256, 3)       225       
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 256, 256, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 256, 256, 3)      0         
 bda)                                                            
                                                                 
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 flatten_1 (Flatten)         (None, 131072)            0 

### Training SRM and Dense Layers

In [52]:
def train_model(model, train_ds, val_ds, max_epochs):
    history = model.fit(train_ds,
                        epochs = max_epochs,
                        validation_data = val_ds,
                        callbacks = [model_checkpoint_callback, 
                                     early_stopping_callback,
                                     model_restore_callback,
                                     MetricsCheckPoint()])
    
    return history, model

In [ ]:
history, SRM_Model = train_model(SRM_Model, train_dataset_cdfv1, val_dataset_cdfv1, 2)

### Saving & Loading Model

In [22]:
SRM_Model.save_weights(SAVE_MODEL_PATH + 'model')

In [23]:
SRM_Model.load_weights(SAVE_MODEL_PATH + 'model')

## Testing Logic

In [39]:
SRM_Model.layers[1].get_weights()

[array([2.0001214], dtype=float32),
 array([3.9997723], dtype=float32),
 array([12.0004425], dtype=float32)]

In [84]:
tmp_srm_model = keras.Sequential([
    keras.layers.Input(shape=(256, 256, 3)),
    SRMLayer(),
    # XceptionPreProcessor(),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation='sigmoid')
])

tmp_srm_model.compile(optimizer=keras.optimizers.Adam(),
                      loss=keras.losses.BinaryCrossentropy(),
                      metrics=['acc'])

tmp_srm_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 srm_layer_17 (SRMLayer)     (None, 256, 256, 3)       3         
                                                                 
 flatten_14 (Flatten)        (None, 196608)            0         
                                                                 
 dense_22 (Dense)            (None, 1)                 196609    
                                                                 
Total params: 196,612
Trainable params: 196,612
Non-trainable params: 0
_________________________________________________________________


In [7]:
tmp_srm_model.layers[0].get_weights()

[array([2.], dtype=float32),
 array([4.], dtype=float32),
 array([12.], dtype=float32)]

In [ ]:
tmp_srm_model.fit(train_dataset_cdfv1, epochs=1, validation_data=val_dataset_cdfv1)

In [9]:
tmp_srm_model.layers[0].get_weights()

[array([2.0912213], dtype=float32),
 array([4.1425376], dtype=float32),
 array([12.13928], dtype=float32)]

In [14]:
tmp_srm_model.layers[2].get_weights()

[array([[-0.00065065],
        [-0.00062465],
        [-0.00713642],
        ...,
        [-0.00407955],
        [ 0.00504078],
        [ 0.00208746]], dtype=float32),
 array([-0.0025801], dtype=float32)]

In [12]:
tmp_srm_model.save_weights('models/tmp_srm_model')

In [15]:
tmp_srm_model_2 = keras.Sequential([
    keras.layers.Input(shape=(256, 256, 3)),
    SRMLayer(),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation='sigmoid')
])

tmp_srm_model.compile(optimizer=keras.optimizers.Adam(),
                      loss=keras.losses.BinaryCrossentropy(),
                      metrics=['acc'])

tmp_srm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 srm_layer (SRMLayer)        (None, 256, 256, 3)       3         
                                                                 
 flatten (Flatten)           (None, 196608)            0         
                                                                 
 dense (Dense)               (None, 1)                 196609    
                                                                 
Total params: 196,612
Trainable params: 196,612
Non-trainable params: 0
_________________________________________________________________


In [19]:
tmp_srm_model_2.layers[0].get_weights()

[array([2.0912213], dtype=float32),
 array([4.1425376], dtype=float32),
 array([12.13928], dtype=float32)]

In [18]:
tmp_srm_model_2.load_weights('models/tmp_srm_model')

In [4]:
tmp_filter_small = tf.constant([[0, 0,  0, 0, 0],
                                [0, 0,  0, 0, 0],
                                [0, 1, -2, 1, 0],
                                [0, 0,  0, 0, 0],
                                [0, 0,  0, 0, 0]], dtype=tf.float32)

tmp_filter_med = tf.constant([[0,  0,  0,  0, 0],
                              [0, -1,  2, -1, 0],
                              [0,  2, -4,  2, 0],
                              [0, -1,  2, -1, 0],
                              [0,  0,  0,  0, 0]], dtype=tf.float32)

tmp_filter_small = tf.expand_dims(tf.expand_dims(tmp_filter_small, axis=-1), axis=-1)
tmp_filter_med   = tf.expand_dims(tf.expand_dims(tmp_filter_med,   axis=-1), axis=-1)
# tf.print(tmp_filter_small.shape)

tmp_filter_small = tf.tile(tmp_filter_small, [1, 1, 3, 1])
tmp_filter_med   = tf.tile(tmp_filter_med,   [1, 1, 3, 1])

tmp_filters = tf.concat([tmp_filter_small, tmp_filter_med], axis=3)

tf.print(tmp_filters.shape)

tmp_conv = Conv2D(filters=2, 
                  kernel_size=5, 
                  kernel_initializer=tf.keras.initializers.Constant(tmp_filters))

tmp_x = tf.random.normal([32, 256, 256, 3])
tmp_y = tmp_conv(tmp_x)

print(tmp_y.shape)

TensorShape([5, 5, 3, 2])
(32, 252, 252, 2)


### Custom SRM Layer

In [9]:
class SSRM(Layer):
    def __init__(self, **kwargs):
        super(SSRM, self).__init__(**kwargs)

        self.filter_small = tf.constant([[0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 1, -2, 1, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0]], dtype=tf.float32)
        
        self.filter_med = tf.constant([[0,  0,  0,  0, 0],
                                       [0, -1,  2, -1, 0],
                                       [0,  2, -4,  2, 0],
                                       [0, -1,  2, -1, 0],
                                       [0,  0,  0,  0, 0]], dtype=tf.float32)
        
        self.filter_large = tf.constant([[-1,  2,  -2,  2, -1],
                                         [ 2, -6,   8, -6,  2],
                                         [-2,  8, -12,  8, -2],
                                         [ 2, -6,   8, -6,  2],
                                         [-1,  2,  -2,  2, -1]], dtype=tf.float32)
        
        self.filter_small = tf.expand_dims(tf.expand_dims(self.filter_small, axis=-1), axis=-1)
        self.filter_med   = tf.expand_dims(tf.expand_dims(self.filter_med,   axis=-1), axis=-1)
        self.filter_large = tf.expand_dims(tf.expand_dims(self.filter_large, axis=-1), axis=-1)

        self.filter_small = tf.tile(self.filter_small, [1, 1, 3, 1])
        self.filter_med   = tf.tile(self.filter_med,   [1, 1, 3, 1])
        self.filter_large = tf.tile(self.filter_large, [1, 1, 3, 1])
        
        def call(self, inputs):
            output_small = tf.nn.conv2d(inputs, self.filter_small, strides=[1, 1, 1, 1], padding='SAME')
            output_med   = tf.nn.conv2d(inputs, self.filter_med,   strides=[1, 1, 1, 1], padding='SAME')
            output_large = tf.nn.conv2d(inputs, self.filter_large, strides=[1, 1, 1, 1], padding='SAME')

            return tf.concat([output_small, output_med, output_large], axis=3)

In [17]:
class PSRM_Small(Layer):
    def __init__(self):
        super(PSRM_Small, self).__init__()

        self.filter_small = tf.constant([[0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 1, -2, 1, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0]], dtype=tf.float32)
        
        self.filter_small = tf.stack([self.filter_small, self.filter_small, self.filter_small], axis=2)
        self.filter_small = tf.expand_dims(self.filter_small, axis=-1)

        self.q_small = self.add_weight(name = 'q_small',
                                 shape = (1, ),
                                 initializer = tf.initializers.Constant(value=2),
                                 trainable = True)
        
    def call(self, inputs):
        filter_small = tf.math.divide(self.filter_small, self.q_small)
        return tf.nn.conv2d(inputs, filter_small, strides=[1,1,1,1], padding='SAME')
        

In [19]:
PSRM_Model = keras.Sequential([
    keras.layers.Input(shape=(256,256,3)),
    PSRM_Small(),
    Flatten(),
    Dense(1, activation='sigmoid')
])

PSRM_Model.compile(optimizer=keras.optimizers.Adam(),
                   loss=keras.losses.BinaryCrossentropy(),
                   metrics=['acc'])

PSRM_Model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 psrm__small_6 (PSRM_Small)  (None, 256, 256, 1)       1         
                                                                 
 flatten_7 (Flatten)         (None, 65536)             0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65537     
                                                                 
Total params: 65,538
Trainable params: 65,538
Non-trainable params: 0
_________________________________________________________________


In [20]:
PSRM_Model.fit(train_dataset_cdfv1, epochs=1)

138/138 [==============================] - 72s 483ms/step - loss: 6.8667 - acc: 0.5918


### Checking Custom Dense Layer

In [10]:
class TMPDense(keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(TMPDense, self).__init__()

        self.w = self.add_weight(name = 'w',
                                 shape = (input_dim, units),
                                 initializer = tf.initializers.random_normal,
                                 trainable = True)
        
        self.b = self.add_weight(name = 'b',
                                  shape = (units, ),
                                  initializer = tf._initializers.zeros,
                                  trainable = True)
        
        self.m = self.add_weight(name = 'm',
                                 shape = (1, ),
                                 initializer = tf.initializers.Constant(value=2.0),
                                 trainable=True)
        
    def call(self, inputs):
        div = tf.math.divide(self.w, self.m)
        return tf.matmul(inputs, div) + self.b

In [12]:
dModel = keras.Sequential([
    keras.layers.Input(shape=(256,256,3)),
    SSRM(),
    Flatten(),
    TMPDense(5, 196608),
    TMPDense(1, 5),
])

dModel.compile(optimizer = keras.optimizers.Adam(),
               loss = keras.losses.BinaryCrossentropy(),
               metrics = ['accuracy']
               )

dModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ssrm_1 (SSRM)               (None, 256, 256, 3)       0         
                                                                 
 flatten_1 (Flatten)         (None, 196608)            0         
                                                                 
 tmp_dense_2 (TMPDense)      (None, 5)                 983046    
                                                                 
 tmp_dense_3 (TMPDense)      (None, 1)                 7         
                                                                 
Total params: 983,053
Trainable params: 983,053
Non-trainable params: 0
_________________________________________________________________


In [13]:
dModel.layers[3].get_weights()

[array([[-0.00287271],
        [-0.04635007],
        [ 0.03803451],
        [-0.02399216],
        [-0.03421838]], dtype=float32),
 array([0.], dtype=float32),
 array([2.], dtype=float32)]

In [17]:
dModel.fit(train_dataset_cdfv1, epochs=5, validation_data=val_dataset_cdfv1)

Epoch 1/5
138/138 [==============================] - 23s 131ms/step - loss: 5.1639 - accuracy: 0.6650 - val_loss: 5.1884 - val_accuracy: 0.6636
Epoch 2/5
138/138 [==============================] - 10s 69ms/step - loss: 5.1708 - accuracy: 0.6648 - val_loss: 5.1884 - val_accuracy: 0.6636
Epoch 3/5
138/138 [==============================] - 16s 113ms/step - loss: 5.1708 - accuracy: 0.6648 - val_loss: 5.1884 - val_accuracy: 0.6636
Epoch 4/5
138/138 [==============================] - 12s 82ms/step - loss: 5.1708 - accuracy: 0.6648 - val_loss: 5.1884 - val_accuracy: 0.6636
Epoch 5/5
138/138 [==============================] - 11s 76ms/step - loss: 5.1708 - accuracy: 0.6648 - val_loss: 5.1884 - val_accuracy: 0.6636


In [18]:
dModel.layers[3].get_weights()

[array([[-0.01285018],
        [-0.05632751],
        [ 0.04801176],
        [-0.01401469],
        [-0.0242409 ]], dtype=float32),
 array([-0.00997706], dtype=float32),
 array([2.0099752], dtype=float32)]

### Custom Conv2D Layer

In [7]:
class TMPConv(Layer):
    def __init__(self):
        super(TMPConv, self).__init__()

        # self.kernel = self.add_weight(name = 'kernel', 
        #                               shape = (3, 3, 3, 1),
        #                               initializer = tf.initializers.random_normal,
        #                               trainable = True)

        self.kernel = tf.constant([[0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 1, -2, 1, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0]], dtype=tf.float32)
        
        self.kernel = tf.stack([self.kernel, self.kernel, self.kernel], axis=2)
        self.kernel = tf.expand_dims(self.kernel, axis=-1)
        
        self.q = self.add_weight(name = 'q',
                                 shape = (1, ),
                                 initializer = tf.initializers.Constant(value=2),
                                 trainable = True)
        
    def call(self, inputs):
        kernel = tf.math.divide(self.kernel, self.q)
        return tf.nn.conv2d(inputs, kernel, strides = [1,1,1,1], padding='SAME')

In [8]:
tmp_model = keras.Sequential([
    keras.layers.Input(shape=(256, 256, 3)),
    TMPConv(),
    Flatten(),
    Dense(1, activation='sigmoid'),
])

tmp_model.compile(loss=keras.losses.BinaryCrossentropy(),
                  optimizer=Adam(),
                  metrics=['acc'])

tmp_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tmp_conv (TMPConv)          (None, 256, 256, 1)       1         
                                                                 
 flatten_1 (Flatten)         (None, 65536)             0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65537     
                                                                 
Total params: 65,538
Trainable params: 65,538
Non-trainable params: 0
_________________________________________________________________


In [58]:
tmp_model.layers[0].get_weights()

[array([2.], dtype=float32)]

In [59]:
tmp_model.fit(train_dataset_cdfv1, epochs=1, validation_data=val_dataset_cdfv1)

138/138 [==============================] - 28s 199ms/step - loss: 5768.4531 - acc: 0.5832 - val_loss: 5201.2295 - val_acc: 0.3682


In [60]:
tmp_model.layers[0].get_weights()

[array([2.0662959], dtype=float32)]

### Old

In [15]:
class TSRM(Layer):
    def __init__(self, **kwargs):
        super(TSRM, self).__init__(**kwargs)

        self.filter_small = tf.constant([[0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 1, -2, 1, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0]], dtype=tf.float32)
        
        self.q_small = self.add_weight(shape=(),
                                       initializer='ones',
                                       dtype=tf.float32,
                                       trainable=True)
        
    def call(self, inputs):
        filter_small = self.filter_small / self.q_small

        # Input shape: [batch_size, height, width, channels]
        # Filter shape: [height, width, in_channels, out_channels]

        conv_small = tf.nn.conv2d(inputs, 
                                  tf.expand_dims(tf.expand_dims(filter_small, axis=-1), axis=-1), 
                                  strides=[1, 1, 1, 1], 
                                  padding='SAME')
        
        # conv_meddd = tf.nn.conv2d(inputs, filter_small, strides=[1, 1, 1, 1], padding='SAME')

        # outputs = tf.concat([conv_small, conv_meddd], axis=-1)
        return conv_small

In [ ]:
tmp_model = keras.Sequential([
    keras.layers.Input(shape=(256, 256, 3)),
    TSRM(),
    Flatten(),
    Dense(1, activation='sigmoid')
])

tmp_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
tmp_history = tmp_model.fit(train_dataset_cdfv1, epochs=1, validation_data=val_dataset_cdfv1)

138/138 [==============================] - 21s 133ms/step - loss: 722.0818 - accuracy: 0.5884 - val_loss: 549.3903 - val_accuracy: 0.4255


In [34]:
tmp_kernel1 = np.array([[0, 0,  0, 0, 0],
                    [0, 0,  0, 0, 0],
                    [0, 1, -2, 1, 0],
                    [0, 0,  0, 0, 0],
                    [0, 0,  0, 0, 0]], dtype=float)

tmp_kernel2 = np.array([[0,  0,  0,  0, 0],
                    [0, -1,  2, -1, 0],
                    [0,  2, -4,  2, 0],
                    [0, -1,  2, -1, 0],
                    [0,  0,  0,  0, 0]], dtype=float)

tmp_kernel3 = np.array([[-1,  2,  -2,  2, -1],
                    [ 2, -6,   8, -6,  2],
                    [-2,  8, -12,  8, -2],
                    [ 2, -6,   8, -6,  2],
                    [-1,  2,  -2,  2, -1]], dtype=float)

tmp_fixed_kernals = [tmp_kernel1, tmp_kernel2, tmp_kernel3]
tmp_custom_layer = SRMLayer(filters=3, kernel_size=(5, 5), fixed_filters=tmp_fixed_kernals)

# Test Video Scores

In [24]:
class VideoScore():
    def __init__(self, filename, score, true_class, fake_on_lower_half = True):
        self.filename = filename
        self.score = score
        self.true_class = true_class
        self.fake_on_lower_half = fake_on_lower_half

    def get_filename(self):
        return self.filename
    
    def get_score(self):
        return self.score
    
    def get_true_class(self):
        return self.true_class
    
    def get_fake_on_lower_half(self):
        return self.fake_on_lower_half

In [25]:
def get_video_score(model, fp):
    frames = extract_snippets(fp, num_segments=8, num_snippets=1)

    tf_frames = []
    for frame in frames:
        tf_frames.append(tf.convert_to_tensor(frame))

    tf_frames = tf.convert_to_tensor(tf_frames)
    results = model.predict(tf_frames, verbose=0)

    return np.average(results)

In [31]:
def get_test_scores(dataset, model):
    video_scores = []

    for class_dir in CLASS:
        print(f'Class started: {class_dir}')
    
        vid_dir = dataset + DS_FACE + DS_TEST + class_dir
        for video in os.listdir(vid_dir):
            fp = vid_dir + video

            video_score = VideoScore(video, 
                                    get_video_score(model, fp),
                                    class_dir)
            
            video_scores.append(video_score)

    return video_scores

In [32]:
SRM_Model_test_scores = get_test_scores(DS_CDFV2, SRM_Model)

with open(TEST_SCORE_PATH + 'srm_scores', 'wb') as f:
    pickle.dump(SRM_Model_test_scores, f)

Class started: real/
Class started: fake/


In [ ]:
SRM_Model_FT_test_scores = get_test_scores(DS_CDFV2, SRM_Model_finetuned)

with open(TEST_SCORE_PATH + 'mesonet_scores', 'wb') as f:
    pickle.dump(SRM_Model_FT_test_scores, f)